# SQL Practice

The data is in the "country_club" database. This database
contains 3 tables:
    - the "Bookings" table,
    - the "Facilities" table, and
    - the "Members" table.

There 10 questions for this case study.


In [ ]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("sqlite_db_pythonsqlite.db")

In [3]:
cur = conn.cursor()

#### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

    SELECT name
    FROM Facilities
    WHERE membercost > 0;

In [4]:
query1 = """
    SELECT name 
    FROM Facilities 
    WHERE membercost > 0;
    """
pd.read_sql_query(query1, conn)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


#### Q2: How many facilities do not charge a fee to members?

    SELECT Count(name)
    FROM Facilities
    WHERE membercost = 0;

In [5]:
query2 = """
    SELECT Count(name) 
    FROM Facilities 
    WHERE membercost = 0;
    """
pd.read_sql_query(query2, conn)

,Count(name)
0,4


#### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

    SELECT facid, name, membercost, monthlymaintenance 
    FROM Facilities 
    WHERE membercost != 0 
        AND membercost < 0.20 * monthlymaintenance;

In [6]:
query3 = """
    SELECT facid, name, membercost, monthlymaintenance 
    FROM Facilities 
    WHERE membercost != 0 
        AND membercost < 0.20 * monthlymaintenance 
    """
pd.read_sql_query(query3, conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


#### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

    SELECT *
    FROM Facilities
    WHERE facid
        IN('1',  '5')

In [7]:
query4 = """
    SELECT * 
    FROM Facilities 
    WHERE facid 
        IN('1',  '5');
    """
pd.read_sql_query(query4, conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


#### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

    SELECT name, monthlymaintenance,
    CASE WHEN monthlymaintenance <= 100 THEN 'Cheap'
         ELSE 'Expensive'
         END AS label
    FROM Facilities

In [8]:
query5 = """
    SELECT name, monthlymaintenance, 
        CASE WHEN monthlymaintenance <= 100 THEN 'Cheap' 
            ELSE 'Expensive' END AS label 
    FROM Facilities;
    """
pd.read_sql_query(query5, conn)

,name,monthlymaintenance,label
0,Tennis Court 1,200,Expensive
1,Tennis Court 2,200,Expensive
2,Badminton Court,50,Cheap
3,Table Tennis,10,Cheap
4,Massage Room 1,3000,Expensive
5,Massage Room 2,3000,Expensive
6,Squash Court,80,Cheap
7,Snooker Table,15,Cheap
8,Pool Table,15,Cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

    SELECT firstname, surname
    FROM Members
    WHERE joindate 
        IN(
            SELECT MAX(joindate) 
            FROM Members
            )

In [9]:
query6 = """
    SELECT firstname, surname
    FROM Members
    WHERE joindate 
        IN(
            SELECT MAX(joindate) 
            FROM Members
            )
    """
pd.read_sql_query(query6, conn)

,firstname,surname
0,Darren,Smith


#### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

    SELECT DISTINCT concat(m.firstname, " ", m.surname)AS member,
        f.name AS facility
    FROM Members AS m 
    INNER JOIN Bookings AS b
    ON m.memid = b.memid
    INNER JOIN Facilities AS f
    ON f.facid = b.facid
    WHERE f.facid 
        IN('0', '1')

In [10]:
query7 = """
    SELECT DISTINCT m.firstname ||\" \" || m.surname AS member,
        f.name AS facility
    FROM Members AS m 
    INNER JOIN Bookings AS b
    ON m.memid = b.memid
    INNER JOIN Facilities AS f
    ON f.facid = b.facid
    WHERE f.facid 
        IN('0', '1');
    """ 

pd.read_sql_query(query7, conn).T

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
member,Tracy Smith,GUEST GUEST,GUEST GUEST,Tim Rownam,Tim Rownam,Darren Smith,Janice Joplette,Gerald Butters,Janice Joplette,Tracy Smith,...,Jemima Farrell,David Farrell,Millicent Purview,Henrietta Rumney,Florence Bader,John Hunt,John Hunt,David Farrell,Matthew Genting,Erica Crumpet
facility,Tennis Court 1,Tennis Court 2,Tennis Court 1,Tennis Court 2,Tennis Court 1,Tennis Court 2,Tennis Court 1,Tennis Court 1,Tennis Court 2,Tennis Court 2,...,Tennis Court 1,Tennis Court 1,Tennis Court 2,Tennis Court 2,Tennis Court 1,Tennis Court 1,Tennis Court 2,Tennis Court 2,Tennis Court 1,Tennis Court 1


#### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

    SELECT 
      DISTINCT concat(m.firstname," ", m.surname) AS member, 
      f.name, f.membercost, f.guestcost,
      CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots END AS cost 
    FROM Members AS m 
    INNER JOIN Bookings AS b
    ON m.memid = b.memid
    INNER JOIN Facilities AS f
    ON f.facid = b.facid
    WHERE date(b.starttime) = '2012-09-14' 
        AND (CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                  ELSE f.membercost * b.slots END) > 30
    ORDER BY cost DESC

In [11]:
query8 = """
    SELECT 
      DISTINCT m.firstname ||\" \" || m.surname AS member, 
      f.name, f.membercost, f.guestcost,
      CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots END AS cost 
    FROM Members AS m 
    INNER JOIN Bookings AS b
    ON m.memid = b.memid
    INNER JOIN Facilities AS f
    ON f.facid = b.facid
    WHERE date(b.starttime) = '2012-09-14' 
        AND (CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                  ELSE f.membercost * b.slots END) > 30
    ORDER BY cost DESC;
    """
pd.read_sql_query(query8, conn)

,member,name,membercost,guestcost,cost
0,GUEST GUEST,Massage Room 2,9.9,80.0,320.0
1,GUEST GUEST,Massage Room 1,9.9,80.0,160.0
2,GUEST GUEST,Tennis Court 2,5.0,25.0,150.0
3,GUEST GUEST,Tennis Court 1,5.0,25.0,75.0
4,GUEST GUEST,Tennis Court 2,5.0,25.0,75.0
5,GUEST GUEST,Squash Court,3.5,17.5,70.0
6,Jemima Farrell,Massage Room 1,9.9,80.0,39.6
7,GUEST GUEST,Squash Court,3.5,17.5,35.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

    SELECT *
    FROM(
      SELECT 
        DISTINCT concat(m.firstname," ", m.surname) AS member, 
        f.name, f.membercost, f.guestcost,
        CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
              ELSE f.membercost * b.slots END AS cost 
      FROM Members AS m 
      INNER JOIN Bookings AS b
      ON m.memid = b.memid
      INNER JOIN Facilities AS f
      ON f.facid = b.facid
      WHERE DATE(b.starttime) = '2012-09-14') AS a
    WHERE a.cost > 30
    ORDER BY a.cost DESC


In [12]:
query9 = """
    SELECT *
    FROM(
      SELECT 
        DISTINCT m.firstname ||\" \" || m.surname AS member, 
        f.name, f.membercost, f.guestcost,
        CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
              ELSE f.membercost * b.slots END AS cost 
      FROM Members AS m 
      INNER JOIN Bookings AS b
      ON m.memid = b.memid
      INNER JOIN Facilities AS f
      ON f.facid = b.facid
      WHERE DATE(b.starttime) = '2012-09-14') AS a
    WHERE a.cost > 30
    ORDER BY a.cost desc
"""
pd.read_sql_query(query9, conn)

,member,name,membercost,guestcost,cost
0,GUEST GUEST,Massage Room 2,9.9,80.0,320.0
1,GUEST GUEST,Massage Room 1,9.9,80.0,160.0
2,GUEST GUEST,Tennis Court 2,5.0,25.0,150.0
3,GUEST GUEST,Tennis Court 1,5.0,25.0,75.0
4,GUEST GUEST,Tennis Court 2,5.0,25.0,75.0
5,GUEST GUEST,Squash Court,3.5,17.5,70.0
6,Jemima Farrell,Massage Room 1,9.9,80.0,39.6
7,GUEST GUEST,Squash Court,3.5,17.5,35.0


#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

     SELECT a.name, SUM(cost) AS total_revenue
        FROM 
           (SELECT f.name,
            CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                 ELSE f.membercost * b.slots END AS cost 
            FROM Members AS m 
            INNER JOIN Bookings AS b
            ON m.memid = b.memid
            INNER JOIN Facilities AS f
            ON f.facid = b.facid) AS a
        GROUP BY a.name having SUM(cost) < 1000
        ORDER BY total_revenue

In [13]:
query10 = """
     SELECT a.name, SUM(cost) AS total_revenue
        FROM 
           (SELECT f.name,
            CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                 ELSE f.membercost * b.slots END AS cost 
            FROM Members AS m 
            INNER JOIN Bookings AS b
            ON m.memid = b.memid
            INNER JOIN Facilities AS f
            ON f.facid = b.facid) AS a
        GROUP BY a.name having SUM(cost) < 1000
        ORDER BY total_revenue
"""
pd.read_sql_query(query10, conn)

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270
